# Human numbers

In [1]:
%load_ext Ddip
%makedip -a fastai_v1 -g all

Waiting for connection file: ~/.ipython/profile_default/security/ipcontroller-ippdpp_c-client.json


In [2]:
%%dip everywhere
from fastai.text import *

In [3]:
bs=64

## Data

In [4]:
path = untar_data(URLs.HUMAN_NUMBERS)
path.ls()

[PosixPath('/home/ndim1/.fastai/data/human_numbers/valid.txt'),
 PosixPath('/home/ndim1/.fastai/data/human_numbers/tmp'),
 PosixPath('/home/ndim1/.fastai/data/human_numbers/train.txt')]

In [5]:
def readnums(d): return [', '.join(o.strip() for o in open(path/d).readlines())]

In [6]:
train_txt = readnums('train.txt'); train_txt[0][:80]

'one, two, three, four, five, six, seven, eight, nine, ten, eleven, twelve, thirt'

In [7]:
valid_txt = readnums('valid.txt'); valid_txt[0][-80:]

' nine thousand nine hundred ninety eight, nine thousand nine hundred ninety nine'

In [8]:
%dipush bs path train_txt valid_txt
%autodip on

In [10]:
train = TextList(train_txt, path=path)
valid = TextList(valid_txt, path=path)

src = ItemLists(path=path, train=train, valid=valid).label_for_lm()
data = src.databunch(bs=bs)

In [11]:
train[0].text[:80]

Out[0:3]: 'xxbos one , two , three , four , five , six , seven , eight , nine , ten , eleve'

In [12]:
len(data.valid_ds[0][0].data)

Out[0:4]: 13017

In [13]:
data.bptt, len(data.valid_dl)

Out[0:5]: (70, 3)

In [14]:
13017/70/bs

Out[0:6]: 2.905580357142857

In [15]:
it = iter(data.valid_dl)
x1,y1 = next(it)
x2,y2 = next(it)
x3,y3 = next(it)
it.close()

In [16]:
x1.numel()+x2.numel()+x3.numel()

Out[0:8]: 13440

In [17]:
x1.shape,y1.shape

Out[0:9]: (torch.Size([64, 70]), torch.Size([64, 70]))

In [18]:
x2.shape,y2.shape

Out[0:10]: (torch.Size([64, 70]), torch.Size([64, 70]))

In [19]:
x1[:,0]

Out[0:11]: 
tensor([ 2, 36, 19,  9, 20, 26, 11, 19, 12, 12, 19,  9, 13, 25, 10, 11, 19, 11,
        10, 13, 19,  9, 14, 25, 10, 11, 19, 14, 33, 14, 19,  9, 15, 25, 10, 11,
        19, 10,  9, 15, 19,  9, 16, 25, 10, 11, 19, 16, 19, 16, 19,  9, 17, 25,
        10, 11, 19,  9, 11, 17, 19,  9, 18, 25], device='cuda:0')

In [20]:
y1[:,0]

Out[0:12]: 
tensor([19,  9, 11, 19,  9, 14, 27, 11, 10, 10, 11, 19,  9, 13, 26, 12, 11, 13,
        32, 10, 11, 19,  9, 14, 26, 13, 11, 10,  9, 10, 11, 19,  9, 15, 26, 14,
        11, 15, 19, 10, 11, 19,  9, 16, 26, 15, 11,  9, 11, 10, 11, 19,  9, 17,
        26, 16, 11, 19, 17, 10, 11, 19,  9, 18], device='cuda:0')

In [22]:
v = data.valid_ds.vocab

In [23]:
v.textify(x1[0])

Out[0:15]: 'xxbos eight thousand one , eight thousand two , eight thousand three , eight thousand four , eight thousand five , eight thousand six , eight thousand seven , eight thousand eight , eight thousand nine , eight thousand ten , eight thousand eleven , eight thousand twelve , eight thousand thirteen , eight thousand fourteen , eight thousand fifteen , eight thousand sixteen , eight thousand seventeen , eight'

In [24]:
v.textify(y1[0])

Out[0:16]: 'eight thousand one , eight thousand two , eight thousand three , eight thousand four , eight thousand five , eight thousand six , eight thousand seven , eight thousand eight , eight thousand nine , eight thousand ten , eight thousand eleven , eight thousand twelve , eight thousand thirteen , eight thousand fourteen , eight thousand fifteen , eight thousand sixteen , eight thousand seventeen , eight thousand'

In [25]:
v.textify(x2[0])

Out[0:17]: 'hundred seventy seven , eight thousand six hundred seventy eight , eight thousand six hundred seventy nine , eight thousand six hundred eighty , eight thousand six hundred eighty one , eight thousand six hundred eighty two , eight thousand six hundred eighty three , eight thousand six hundred eighty four , eight thousand six hundred eighty five , eight thousand six hundred eighty six , eight thousand six hundred'

In [26]:
v.textify(x3[0])

Out[0:18]: 'fifty four , nine thousand three hundred fifty five , nine thousand three hundred fifty six , nine thousand three hundred fifty seven , nine thousand three hundred fifty eight , nine thousand three hundred fifty nine , nine thousand three hundred sixty , nine thousand three hundred sixty one , nine thousand three hundred sixty two , nine thousand three hundred sixty three , nine thousand three hundred sixty'

In [27]:
v.textify(x1[1])

Out[0:19]: 'seventeen , eight thousand eighteen , eight thousand nineteen , eight thousand twenty , eight thousand twenty one , eight thousand twenty two , eight thousand twenty three , eight thousand twenty four , eight thousand twenty five , eight thousand twenty six , eight thousand twenty seven , eight thousand twenty eight , eight thousand twenty nine , eight thousand thirty , eight thousand thirty one , eight thousand'

In [28]:
v.textify(x2[1])

Out[0:20]: 'thousand six hundred eighty seven , eight thousand six hundred eighty eight , eight thousand six hundred eighty nine , eight thousand six hundred ninety , eight thousand six hundred ninety one , eight thousand six hundred ninety two , eight thousand six hundred ninety three , eight thousand six hundred ninety four , eight thousand six hundred ninety five , eight thousand six hundred ninety six , eight thousand'

In [29]:
v.textify(x3[1])

Out[0:21]: 'three hundred sixty four , nine thousand three hundred sixty five , nine thousand three hundred sixty six , nine thousand three hundred sixty seven , nine thousand three hundred sixty eight , nine thousand three hundred sixty nine , nine thousand three hundred seventy , nine thousand three hundred seventy one , nine thousand three hundred seventy two , nine thousand three hundred seventy three , nine thousand three'

In [30]:
v.textify(x3[-1])

Out[0:22]: 'nine thousand nine hundred ninety , nine thousand nine hundred ninety one , nine thousand nine hundred ninety two , nine thousand nine hundred ninety three , nine thousand nine hundred ninety four , nine thousand nine hundred ninety five , nine thousand nine hundred ninety six , nine thousand nine hundred ninety seven , nine thousand nine hundred ninety eight , nine thousand nine hundred ninety nine xxbos eight'

In [31]:
data.show_batch(ds_type=DatasetType.Valid)

[output:0]

idx,text
0,"thousand eighteen , eight thousand nineteen , eight thousand twenty , eight thousand twenty one , eight thousand twenty two , eight thousand twenty three , eight thousand twenty four , eight thousand twenty five , eight thousand twenty six , eight thousand twenty seven , eight thousand twenty eight , eight thousand twenty nine , eight thousand thirty , eight thousand thirty one , eight thousand thirty two ,"
1,"thirty two , eight thousand thirty three , eight thousand thirty four , eight thousand thirty five , eight thousand thirty six , eight thousand thirty seven , eight thousand thirty eight , eight thousand thirty nine , eight thousand forty , eight thousand forty one , eight thousand forty two , eight thousand forty three , eight thousand forty four , eight thousand forty five , eight thousand forty"
2,"thousand forty six , eight thousand forty seven , eight thousand forty eight , eight thousand forty nine , eight thousand fifty , eight thousand fifty one , eight thousand fifty two , eight thousand fifty three , eight thousand fifty four , eight thousand fifty five , eight thousand fifty six , eight thousand fifty seven , eight thousand fifty eight , eight thousand fifty nine , eight thousand"
3,"eight thousand sixty , eight thousand sixty one , eight thousand sixty two , eight thousand sixty three , eight thousand sixty four , eight thousand sixty five , eight thousand sixty six , eight thousand sixty seven , eight thousand sixty eight , eight thousand sixty nine , eight thousand seventy , eight thousand seventy one , eight thousand seventy two , eight thousand seventy three , eight thousand"
4,"eight thousand seventy four , eight thousand seventy five , eight thousand seventy six , eight thousand seventy seven , eight thousand seventy eight , eight thousand seventy nine , eight thousand eighty , eight thousand eighty one , eight thousand eighty two , eight thousand eighty three , eight thousand eighty four , eight thousand eighty five , eight thousand eighty six , eight thousand eighty seven , eight"


## Single fully connected model

In [32]:
data = src.databunch(bs=bs, bptt=3)

In [33]:
x,y = data.one_batch()
x.shape,y.shape

Out[0:25]: (torch.Size([64, 3]), torch.Size([64, 3]))

In [34]:
nv = len(v.itos); nv

Out[0:26]: 40

In [35]:
nh=64

In [36]:
def loss4(input,target): return F.cross_entropy(input, target[:,-1])
def acc4 (input,target): return accuracy(input, target[:,-1])

In [37]:
class Model0(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)  # green arrow
        self.h_h = nn.Linear(nh,nh)     # brown arrow
        self.h_o = nn.Linear(nh,nv)     # blue arrow
        self.bn = nn.BatchNorm1d(nh)
        
    def forward(self, x):
        h = self.bn(F.relu(self.h_h(self.i_h(x[:,0]))))
        if x.shape[1]>1:
            h = h + self.i_h(x[:,1])
            h = self.bn(F.relu(self.h_h(h)))
        if x.shape[1]>2:
            h = h + self.i_h(x[:,2])
            h = self.bn(F.relu(self.h_h(h)))
        return self.h_o(h)

In [38]:
learn = Learner(data, Model0(), loss_func=loss4, metrics=acc4)

In [39]:
learn.fit_one_cycle(6, 1e-4)

epoch     train_loss  valid_loss  acc4      time    
0         3.774806    3.665500    0.092844  00:00                                        
1         3.462303    3.256816    0.287364  00:00                                        
2         3.018615    2.863416    0.404438  00:00                                        
3         2.680691    2.637917    0.433424  00:00                                        
4         2.502801    2.556014    0.432971  00:00                                        
5         2.440245    2.543663    0.434556  00:00                                        


## Same thing with a loop

In [40]:
class Model1(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)  # green arrow
        self.h_h = nn.Linear(nh,nh)     # brown arrow
        self.h_o = nn.Linear(nh,nv)     # blue arrow
        self.bn = nn.BatchNorm1d(nh)
        
    def forward(self, x):
        h = torch.zeros(x.shape[0], nh).to(device=x.device)
        for i in range(x.shape[1]):
            h = h + self.i_h(x[:,i])
            h = self.bn(F.relu(self.h_h(h)))
        return self.h_o(h)

In [41]:
learn = Learner(data, Model1(), loss_func=loss4, metrics=acc4)

In [42]:
learn.fit_one_cycle(6, 1e-4)

epoch     train_loss  valid_loss  acc4      time    
0         3.758703    3.691427    0.052536  00:00                                        
1         3.421407    3.223214    0.206748  00:00                                        
2         2.956229    2.799726    0.408967  00:00                                        
3         2.612568    2.570048    0.443614  00:00                                        
4         2.434925    2.485586    0.447237  00:00                                        
5         2.373689    2.473126    0.447917  00:00                                        


## Multi fully connected model

In [43]:
data = src.databunch(bs=bs, bptt=20)

In [44]:
x,y = data.one_batch()
x.shape,y.shape

Out[0:36]: (torch.Size([64, 20]), torch.Size([64, 20]))

In [45]:
class Model2(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.h_h = nn.Linear(nh,nh)
        self.h_o = nn.Linear(nh,nv)
        self.bn = nn.BatchNorm1d(nh)
        
    def forward(self, x):
        h = torch.zeros(x.shape[0], nh).to(device=x.device)
        res = []
        for i in range(x.shape[1]):
            h = h + self.i_h(x[:,i])
            h = F.relu(self.h_h(h))
            res.append(self.h_o(self.bn(h)))
        return torch.stack(res, dim=1)

In [46]:
learn = Learner(data, Model2(), metrics=accuracy)

In [47]:
learn.fit_one_cycle(10, 1e-4, pct_start=0.1)

epoch     train_loss  valid_loss  accuracy  time    
0         3.858204    3.687515    0.029297  00:00                                         
1         3.812659    3.611329    0.035872  00:00                                         
2         3.754122    3.538536    0.045703  00:00                                         
3         3.692020    3.475484    0.070312  00:00                                         
4         3.631007    3.424283    0.087240  00:00                                         
5         3.574345    3.386275    0.092318  00:00                                         
6         3.524519    3.361432    0.114063  00:00                                         
7         3.483077    3.347867    0.117122  00:00                                         
8         3.450490    3.342597    0.118034  00:00                                         
9         3.426121    3.341781    0.118099  00:00                                          


## Maintain state

In [48]:
class Model3(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.h_h = nn.Linear(nh,nh)
        self.h_o = nn.Linear(nh,nv)
        self.bn = nn.BatchNorm1d(nh)
        self.h = torch.zeros(bs, nh).cuda()
        
    def forward(self, x):
        res = []
        h = self.h
        for i in range(x.shape[1]):
            h = h + self.i_h(x[:,i])
            h = F.relu(self.h_h(h))
            res.append(self.bn(h))
        self.h = h.detach()
        res = torch.stack(res, dim=1)
        res = self.h_o(res)
        return res

In [49]:
learn = Learner(data, Model3(), metrics=accuracy)

In [50]:
learn.fit_one_cycle(20, 3e-3)

epoch     train_loss  valid_loss  accuracy  time    
0         3.882783    3.836292    0.053385  00:00                                         
1         3.714467    3.493464    0.125195  00:00                                         
2         3.405393    2.902032    0.230534  00:00                                         
3         2.970771    2.189735    0.406445  00:00                                         
4         2.556883    2.064996    0.318164  00:00                                         
5         2.253943    2.082781    0.316602  00:00                                         
6         2.040581    2.115592    0.316146  00:00                                         
7         1.886860    2.119180    0.319466  00:00                                         
8         1.769891    2.054788    0.353125  00:00                                         
9         1.676100    2.033432    0.399349  00:00                                          
10        1.597509    2.028480    0.

## nn.RNN

In [51]:
class Model4(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.rnn = nn.RNN(nh,nh, batch_first=True)
        self.h_o = nn.Linear(nh,nv)
        self.bn = BatchNorm1dFlat(nh)
        self.h = torch.zeros(1, bs, nh).cuda()
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(self.bn(res))

In [52]:
learn = Learner(data, Model4(), metrics=accuracy)

In [53]:
learn.fit_one_cycle(20, 3e-3)

epoch     train_loss  valid_loss  accuracy  time    
0         3.758616    3.613478    0.012435  00:00                                         
1         3.500630    3.048081    0.348698  00:00                                         
2         3.083210    2.251133    0.457682  00:00                                         
3         2.658812    2.055802    0.462370  00:00                                         
4         2.332969    2.108472    0.317513  00:00                                         
5         2.101010    2.160476    0.315039  00:00                                         
6         1.935602    2.196824    0.316276  00:00                                         
7         1.815481    2.226831    0.315885  00:00                                         
8         1.724959    2.217088    0.317708  00:00                                         
9         1.649180    1.988430    0.351367  00:00                                          
10        1.583829    1.903864    0.

## 2-layer GRU

In [54]:
class Model5(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.rnn = nn.GRU(nh, nh, 2, batch_first=True)
        self.h_o = nn.Linear(nh,nv)
        self.bn = BatchNorm1dFlat(nh)
        self.h = torch.zeros(2, bs, nh).cuda()
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(self.bn(res))

In [55]:
learn = Learner(data, Model5(), metrics=accuracy)

In [56]:
learn.fit_one_cycle(10, 1e-2)

epoch     train_loss  valid_loss  accuracy  time    
0         3.370053    3.395743    0.389648  00:00                                         
1         2.613388    2.065595    0.477148  00:00                                         
2         2.003375    1.606392    0.557617  00:00                                         
3         1.523717    1.373229    0.672917  00:00                                         
4         1.143474    1.226494    0.751042  00:00                                         
5         0.865110    1.075886    0.772201  00:00                                         
6         0.662563    1.117133    0.779753  00:00                                         
7         0.516041    1.090799    0.780599  00:00                                         
8         0.408326    1.092041    0.771745  00:00                                         
9         0.329717    1.084419    0.773242  00:00                                          


## fin